# Word2Vec

In [1]:
import gensim
import wget
import urllib
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer() #сразу же инициализируем морфанализатор
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

## Загрузка готовой векторной модели с сайта RusVectores

Зазгрузим модель ruscorpora_upos_skipgram_600_10_2017, обученную на корпусе НКРЯ.

Ссылка на модель находится [здесь]('https://rusvectores.org/static/models/ruscorpora_upos_skipgram_600_10_2017.bin.gz')

Список всех моделей на сайте RusVectores [тут](https://rusvectores.org/ru/models/) 

**Характеристики модели, которую мы используем в этой тетрадке:**

Дата создания  - октябрь 2017

Размер обучающего корпуса - 250 миллионов слов 

Алгоритм обучения - Continuous Skipgram

Размер окна - 10

Размерность вектора - 600

Набор частеречных тэгов - Universal tags

In [2]:
urllib.request.urlretrieve("https://rusvectores.org/static/models/ruscorpora_upos_skipgram_600_10_2017.bin.gz",
"ruscorpora_upos_skipgram_600_10_2017.bin.gz")


('ruscorpora_upos_skipgram_600_10_2017.bin.gz',
 <http.client.HTTPMessage at 0x122ca4a90>)

In [3]:
# Загружаем модель
model_path = 'ruscorpora_upos_skipgram_600_10_2017.bin.gz'
model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [4]:
# Функция для предобработки и токенизации текста
def preprocessing(text):
    text = re.sub(r'[!\"#$%&\'()*+,./:;<=>?@^_`{|}~]|( -)|(\[)|(\])', '', text) # удаляем пунктуацию
    tokens =  word_tokenize(text)
    return [token for token in tokens if token not in stopwords.words('russian')] #из всех слов убираем стоп-слова

## Создание словаря для конвертации частеречных тэгов Pymorphy2 в формат Unversal POS tags

Частеречные тэги [Pymorphy2](http://opencorpora.org/dict.php?act=gram&order=priority) 

Частеречные тэги [Unversal POS](https://universaldependencies.org/u/pos/all.html)

In [5]:
mapping_Pymorphy_UPos = {
'ADJF': 'ADJ',
'ADJS': 'ADJ',
'ADVB': 'ADV',
'Apro': 'DET',
'COMP': 'ADV',  # может быть и прилагательным, недостаточно информации о тэге в opencorpora
'CONJ': 'CONJ',
'GRND': 'VERB',
'INFN': 'VERB',
'INTJ': 'INTJ',
'NOUN': 'NOUN',
'NPRO': 'PRON',
'NUMR': 'NUM',
'NUMB': 'NUM',
'PART': 'PRCL',
'PNCT': 'PUNCT',
'PRCL': 'PART',
'PREP': 'ADP',
'PRTF': 'VERB',
'PRTS': 'VERB',
'VERB': 'VERB'
}

## Создание функции для лемматизации, частеречной разметки

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). В названиях моделей на rusvectores указано, какой тегсет они используют (mystem, upos и т.д.)

In [6]:
"""Функция лемматизирует текст и осуществляет частеречную разметку в виде Universal POS-tags. 
(Предобработка необходима для работы с предобученной моделью от RusVectores.)


Аргументы:
    tokens - список токенов текста 
    mapping - словарь-таблица для конвертации тэгов частей речи из одного формата в другой.  
Вывод:
    tagged_lemmas - список лемм с частеречным тэгом
'''
"""

def get_lemmas_with_pos_tag(tokens, mapping):
    tagged_lemmas = []
    for token in tokens:
        morph_analized = morph.parse(token) # морф анализ
        lemma = morph_analized[0].normal_form #извлекаем лемму
        #print('Лемма:', lemma)
        pos = morph_analized[0].tag.POS #извлекаем частеречный тэг
        #print('Тэг:', pos)
        #Проверяем, есть ли частеречный тэг в нашем словаре:
        if pos in mapping: 
            tagged_lemma = lemma + '_' + mapping[pos] #получаем лемму с частеречным тэгом: lemma_POS-tag
            tagged_lemmas.append(tagged_lemma)
        else:
            tagged_lemma = lemma + '_X' # на случай, если попадется тэг, которого нет в словаре
            tagged_lemmas.append(tagged_lemma)
        
    return tagged_lemmas

In [7]:
tokens = preprocessing('Папа и отец - слова синонимы и это всем известно, не так ли?')

In [8]:
tokens

['Папа', 'отец', 'слова', 'синонимы', 'это', 'всем', 'известно']

In [10]:
lemmas = get_lemmas_with_pos_tag(tokens, mapping_Pymorphy_UPos)
lemmas

['папа_NOUN',
 'отец_NOUN',
 'слово_NOUN',
 'синоним_NOUN',
 'это_PART',
 'весь_ADJ',
 'известно_X']

## Проведем тест модели

Попросим у модели 10 ближайших соседей и коэффициент косинусной близости  для каждого слова:

In [11]:
def test_word2vec(words):
    for word in words:
        # проверяем, есть ли слово в модели
        if word in model_ru:
            print(word)
            print(model_ru[word][:10])
            #выдаем 10 ближайших соседей слова:
            for word, sim in model_ru.most_similar(positive=[word], topn=10):
                #слово + коэффициент косинусной близости
                print(word, ': ', sim)
            print('\n')
        else:
            print('Увы, слова {} нет в модели!'.format(word))

In [12]:
test_word2vec(lemmas)

папа_NOUN
[-0.01874044 -0.01209912  0.02758167  0.04107445 -0.02311327 -0.06165204
  0.01347432 -0.00246528 -0.02627983 -0.01095002]
мама_NOUN :  0.6132853031158447
сангий_NOUN :  0.5607517957687378
солнечкин_NOUN :  0.5561753511428833
лоли_NOUN :  0.5431061387062073
пидет_ADV :  0.5387659072875977
отец_NOUN :  0.5260175466537476
бонифаций_NOUN :  0.5180080533027649
маин_NOUN :  0.517593264579773
мамочка_NOUN :  0.5093820095062256
баби_NOUN :  0.49594396352767944


отец_NOUN
[-0.01164836 -0.03375478  0.00456757  0.04077563  0.01659235 -0.04487033
  0.03869454 -0.01065875 -0.03143692 -0.01896806]
родитель_NOUN :  0.5874082446098328
нухим_NOUN :  0.5746214985847473
гарклавс_NOUN :  0.5510189533233643
туберозов_NOUN :  0.5497263669967651
иеронимо_NOUN :  0.5477081537246704
мать_NOUN :  0.5472304224967957
логиныч_NOUN :  0.54477858543396
голиндуха_NOUN :  0.5423721075057983
иху_NOUN :  0.5393348932266235
дашенькин_NOUN :  0.5384094715118408


слово_NOUN
[ 0.02531466 -0.00931365 -0.01961394

Находим косинусную близость пары слов:

In [13]:
print(model_ru.similarity('отец_NOUN', 'папа_NOUN'))

0.5260176
